<a href="https://colab.research.google.com/github/jpscard/Pothole-and-Speed-Bump-Classification-using-Yolov7/blob/main/Buracos_e_Lombadas_com_YOLOv7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Train YOLOv7 on a Custom Dataset

This tutorial is based on the [YOLOv7 repository](https://github.com/WongKinYiu/yolov7) by WongKinYiu. This notebook shows training on **your own custom objects**. Many thanks to WongKinYiu and AlexeyAB for putting this repository together.


### **Accompanying Blog Post**

We recommend that you follow along in this notebook while reading the blog post on [how to train YOLOv7](https://blog.roboflow.com/yolov7-custom-dataset-training-tutorial/), concurrently.

### **Steps Covered in this Tutorial**

To train our detector we take the following steps:

* Install YOLOv7 dependencies
* Load custom dataset from Roboflow in YOLOv7 format
* Run YOLOv7 training
* Evaluate YOLOv7 performance
* Run YOLOv7 inference on test images
* OPTIONAL: Deployment
* OPTIONAL: Active Learning


### Preparing a Custom Dataset

In this tutorial, we will utilize an open source computer vision dataset from one of the 90,000+ available on [Roboflow Universe](https://universe.roboflow.com).

If you already have your own images (and, optionally, annotations), you can convert your dataset using [Roboflow](https://roboflow.com), a set of tools developers use to build better computer vision models quickly and accurately. 100k+ developers use roboflow for (automatic) annotation, converting dataset formats (like to YOLOv7), training, deploying, and improving their datasets/models.

Follow [the getting started guide here](https://docs.roboflow.com/quick-start) to create and prepare your own custom dataset.

#Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [ ]:
# MONTAR DRIVE (opcional por enquanto, ainda irei ajustar para baixar do meu gdrive, por enquanto anexem manualmente os pesos e pasta de testes)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download YOLOv7 repository and install requirements
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

# Download Correctly Formatted Custom Data

Next, we'll download our dataset in the right format. Use the `YOLOv7 PyTorch` export. Note that this model requires YOLO TXT annotations, a custom YAML file, and organized directories. The roboflow export writes this for us and saves it in the correct spot.


In [ ]:
#Chamando 2 versão do dataset
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Gp6ex9KXfnqKTX96ivFP")
project = rf.workspace("").project("dataset_buraco_e_lombada")
dataset = project.version(2).download("yolov7")

# **Essa etapa usará os pesos treinados com o dataset do autor como um tranfer learning. Podemos ignorar e colocar nosso proprio peso. na etapa a seguir**

# Begin Custom Training

We're ready to start custom training.

NOTE: We will only modify one of the YOLOv7 training defaults in our example: `epochs`. We will adjust from 300 to 100 epochs in our example for speed. If you'd like to change other settings, see details in [our accompanying blog post](https://blog.roboflow.com/yolov7-custom-dataset-training-tutorial/).

In [ ]:
# download COCO starting checkpoint
%cd /content/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

# **Refazer os treinos sobre nossos proprios pesos**

In [ ]:
# run this cell to begin training
%cd /content/yolov7
!python train.py --batch 16 --epochs 1 --data {dataset.location}/data.yaml --weights '/content/yolov7/runs/train/best.pt' --device 0 


# Evaluation

We can evaluate the performance of our custom training using the provided evalution script.

Note we can adjust the below custom arguments. For details, see [the arguments accepted by detect.py](https://github.com/WongKinYiu/yolov7/blob/main/detect.py#L154).

**Atentar para o caminho correto do nosso melhor peso (best.pt) podemos direcionar o caminho a partir do gdrive, ou fazer o upload manualmente.**

**Da mesma forma, direcionar o caminho da pasta de testes, via gdrive ou manualmente. o resize facilita mas não se faz necessário (preciso confirmar essa questão)**

In [ ]:
# Run evaluation
!python detect.py --weights /content/yolov7/runs/train/best.pt --conf 0.1 --source {dataset.location}/testes #/images


In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

i = 0
limit = 10000 # max images to print
for imageName in glob.glob('/content/yolov7/runs/detect/exp2/*.jpg'): #assuming JPG
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1
    

# Reparameterize for Inference

https://github.com/WongKinYiu/yolov7/blob/main/tools/reparameterization.ipynb

# OPTIONAL: Deployment

To deploy, you'll need to export your weights and save them to use later.

In [ ]:
# optional, zip to download weights and results locally

!zip -r export.zip runs/detect
!zip -r export.zip runs/train/exp/weights/best.pt
!zip export.zip runs/train/exp/*

updating: runs/detect/ (stored 0%)
updating: runs/detect/exp/ (stored 0%)
updating: runs/detect/exp/img-394_jpg.rf.838e7cc63bc43ebe385cc2ab2df05b3a.jpg (deflated 4%)
updating: runs/detect/exp/67_jpg.rf.25a034582cccd1460620b92c1432fce6.jpg (deflated 5%)
updating: runs/detect/exp/96_jpg.rf.1779d99a35ff35c5e55c4bb93089392a.jpg (deflated 4%)
updating: runs/detect/exp/img-430_jpg.rf.49be02ca95ff5eb08bfa953fd9b66c46.jpg (deflated 4%)
updating: runs/detect/exp/img-179_jpg.rf.f5d54c7459a4aab0a68be0836d642836.jpg (deflated 5%)
updating: runs/detect/exp/img-65_jpg.rf.3b8bb6ef8c7a66b715fdb5ea242231d8.jpg (deflated 4%)
updating: runs/detect/exp/img-344_jpg.rf.2d4a50ca96ed913d3f8d593c57a26bbd.jpg (deflated 3%)
updating: runs/detect/exp/img-42_jpg.rf.aa335ef2240825298e990d043a4f6586.jpg (deflated 5%)
updating: runs/detect/exp/img-334_jpg.rf.e3b82eec92fc246cbc645c8e2eb047ab.jpg (deflated 4%)
updating: runs/detect/exp/img-410_jpg.rf.94e9063b13ec3b7176fb117e8322c6dd.jpg (deflated 4%)
updating: runs/det

# OPTIONAL: Active Learning Example

Once our first training run is complete, we should use our model to help identify which images are most problematic in order to investigate, annotate, and improve our dataset (and, therefore, model).

To do that, we can execute code that automatically uploads images back to our hosted dataset if the image is a specific class or below a given confidence threshold.


In [ ]:
# # setup access to your workspace
# rf = Roboflow(api_key="YOUR_API_KEY")                               # used above to load data
# inference_project =  rf.workspace().project("YOUR_PROJECT_NAME")    # used above to load data
# model = inference_project.version(1).model

# upload_project = rf.workspace().project("YOUR_PROJECT_NAME")

# print("inference reference point: ", inference_project)
# print("upload destination: ", upload_project)

In [ ]:
# # example upload: if prediction is below a given confidence threshold, upload it 

# confidence_interval = [10,70]                                   # [lower_bound_percent, upper_bound_percent]

# for prediction in predictions:                                  # predictions list to loop through
#   if(prediction['confidence'] * 100 >= confidence_interval[0] and 
#           prediction['confidence'] * 100 <= confidence_interval[1]):
        
#           # upload on success!
#           print(' >> image uploaded!')
#           upload_project.upload(image, num_retry_uploads=3)     # upload image in question

# Next steps

Congratulations, you've trained a custom YOLOv7 model! Next, start thinking about deploying and [building an MLOps pipeline](https://docs.roboflow.com) so your model gets better the more data it sees in the wild.